In [1]:
import numpy as np
from utils import Util
from rbm import RBM
from IPython.display import display as d

In [2]:
util = Util()
dir = 'etl/'
rows = 5000
ratings, attractions = util.read_data(dir)
ratings = util.clean_subset(ratings, rows)
num_vis = len(ratings)
free_energy = True
train = util.preprocess(ratings)
valid = None
if free_energy:
    train, valid = util.split_data(train)
H = 128
user = 1
alpha = 0.01
w = np.random.normal(loc=0, scale=0.01, size=[num_vis, H])
rbm = RBM(alpha, H, num_vis)
epochs = 20
batch_size = 8

v = True

Reading the data
Extracting num_rows from ratings
Preprocessing the dataset
Free energy required, dividing into train and validation sets
Total size of the data is: 5000
Size of the training data is: 3750
Size of the validation data is: 1250


In [3]:
filename = "e"+str(epochs)+"_r"+str(rows)+"_lr"+str(alpha)+"_hu"+str(H)+"_bs"+str(batch_size)
reco, prv_w, prv_vb, prv_hb = rbm.training(train, valid, user, epochs, batch_size, free_energy, v, filename)
reco_l, prv_w_l, prv_vb_l, prv_hb_l = rbm.load_predict(filename, train, user)

Shape of Training Data: (3750, 4)
Training RBM with 20 epochs and batch size: 8
Starting the training process
Shapes: v=(8, 4), W=(5000, 128), hb=(128,), vb=(5000,)


InvalidArgumentError: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [8,4], In[1]: [5000,128] [Op:MatMul] name: 

In [ ]:
if np.all(prv_w == prv_w_l) and np.all(prv_vb == prv_vb_l) and np.all(prv_hb == prv_hb_l):
    print ("Model saved and loaded successfully")

Model saved and loaded successfully


In [ ]:
unseen, seen = rbm.calculate_scores(ratings, attractions, reco_l, user)
rbm.export(unseen, seen, 'recommendations/'+filename, str(user))